In [1]:
from langchain_community.document_loaders import UnstructuredODTLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceBgeEmbeddings
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
file_path = (
    "content/MartinCV.odt"
)
loader = UnstructuredODTLoader(file_path)
pages = loader.load()

print(len(pages))
print(pages)

1
[Document(metadata={'source': 'content/MartinCV.odt'}, page_content='Contacto\n\nArgentina, Buenos Aires, Punta Alta ● telefono celular: +54 2932 45-8108 Correo electronico email: piampianimartin@gmail.com ● linkedin.com/in/martin-piampiani ● github.com/1Pampu\n\nRESUMEN\n\nGraduado en Programación por la Universidad Provincial del Sudoeste, donde fortalecí mis habilidades técnicas y adquirí una sólida base en desarrollo de software. Con un nivel de inglés B2 certificado, me especializo en el desarrollo Backend y destaco por mi capacidad de adaptación y flexibilidad.\n\n  HABILIDADES TÉCNICAS - TECNOLOGIAS\n\n● Python ● C# ● JavaScript ● Docker ● Django ● .NET Framework ● HTML5 ● Linux ● Flask ● SQL ● CSS ● API REST ● FastAPI ● MySQL ● GIT ● OOP\n\n  FORMACIÓN ACADÉMICA - ESTUDIOS\n\nTecnicatura Universitaria en Tecnologías de la Programación ●Universidad Provincial del Sudoeste 2021-2023\n\nPrograma de estudios enfocado en la iniciación en el desarrollo fullstack, bases de datos, an

In [3]:
spliter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)

chunks = spliter.split_documents(pages)
print(len(chunks))
for i, _ in enumerate(chunks):
    print(f'chunk # {i} len: {len(chunks[i].page_content)}: {chunks[i].page_content}')
print(chunks)

6
chunk # 0 len: 498: Contacto

Argentina, Buenos Aires, Punta Alta ● telefono celular: +54 2932 45-8108 Correo electronico email: piampianimartin@gmail.com ● linkedin.com/in/martin-piampiani ● github.com/1Pampu

RESUMEN

Graduado en Programación por la Universidad Provincial del Sudoeste, donde fortalecí mis habilidades técnicas y adquirí una sólida base en desarrollo de software. Con un nivel de inglés B2 certificado, me especializo en el desarrollo Backend y destaco por mi capacidad de adaptación y flexibilidad.
chunk # 1 len: 480: HABILIDADES TÉCNICAS - TECNOLOGIAS

● Python ● C# ● JavaScript ● Docker ● Django ● .NET Framework ● HTML5 ● Linux ● Flask ● SQL ● CSS ● API REST ● FastAPI ● MySQL ● GIT ● OOP

  FORMACIÓN ACADÉMICA - ESTUDIOS

Tecnicatura Universitaria en Tecnologías de la Programación ●Universidad Provincial del Sudoeste 2021-2023

Programa de estudios enfocado en la iniciación en el desarrollo fullstack, bases de datos, análisis de datos, machine learning y programación

In [4]:
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma.from_documents(
    chunks,
    embedding=hf,
)

d:\02-NextCloud\github\LangChain-Playground\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":2})

retrieved_docs = retriever.invoke("tiene alguna experiencia laboral?")

print(len(retrieved_docs))
for doc in retrieved_docs:
    print(doc.page_content)

2
Upper Intermediate B2 Level ●Instituto de Inglés John H. Newman 2011-2019

Programa completo de inglés que cubre habilidades auditivas, expresión oral, lectura y escritura.

  Experiencia laboral

 Freelance developer 01/2024 – Ahora

 A pesar de no tener experiencia laboral formal, trabajo de forma freelance para pequeños clientes que hice con el tiempo, en su mayoria para realizar web apps.

“No Country” - Simulación Laboral
“No Country” - Simulación Laboral

Participé como desarrollador backend en una de las simulaciones laborales ofrecidas por la startup "No Country". Junto con un equipo, desarrollamos una aplicación completamente funcional desde cero. Durante esta simulación, aplicamos metodologías ágiles, incluyendo reuniones diarias de Scrum y planificación de sprints.

  PROYECTOS

Gestor de contratos

Desarrollo un sistema de gestión de contratos para un cliente, incluyendo la creación de


In [6]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=hf_token
)
llama = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\piamp\.cache\huggingface\token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
sistem_template = """Sos PampuAI creado por martin piampiani para responder preguntas sobre su CV.
Responde unicamente a la pregunta que se te hace unicamente con el contexto que se te proporciona, si no sabes la respuesta solo deci que no sabes la respuesta.
Da una respuesta completa, no omitas cosas, y cuando puedas da ejemplos.
"""

message_template = """
{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("system", sistem_template),("human", message_template)])

In [9]:
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llama | StrOutputParser()
print(chain.invoke(input("")))

Según la información proporcionada, Martin maneja las siguientes tecnologías:

1. Lenguajes de programación:
	* Python
	* C#
	* JavaScript
2. Frameworks y bibliotecas:
	* Django
	*.NET Framework
	* Flask
	* FastAPI
3. Base de datos:
	* SQL
	* MySQL
4. Herramientas de desarrollo:
	* Git
5. Otras tecnologías:
	
